# Level 1
# Client sends 2 of 3 parts of encrypted data to other parties via pickle

## Preparations

In [ ]:
# modules
import socket  # creating server client connections for efficient communication
import pickle  # efficient file saving & loading
import time  # for stopping runtimes
import pandas as pd  # data mangling and transforming
import numpy as np  # matrix manipulations
from datetime import datetime  # for timestamps

In [ ]:
# hostnames & ips
file = open('hosts&ips.txt', 'r')
hoips = eval(file.read())
file.close()

In [ ]:
# read local encrypted data
file = open('data/encryptedData.pickle', 'rb')
d = pickle.load(file)

In [ ]:
d.head()

## Communication with other parties

### Establish connection to other parties (servers)

In [ ]:
# start timing
start = time.time()
# general options
s0 = socket.socket()  # Create socket object 0
s1 = socket.socket()  # Create socket object 1
client = socket.gethostname()  # define current machine as client
port = 60387  # Reserve a port for your service.

In [ ]:
# other parties as servers
servers = [key for key, value in hoips['hosts'].items() if value not in client]
server0 = hoips['hosts'][servers[0]]
server1 = hoips['hosts'][servers[1]]

In [ ]:
# double-check parties
print('Server0 ('+servers[0]+'): '+server0)
print('Server1 ('+servers[1]+'): '+server1)
print('Client: '+client)

In [ ]:
# connect
s0.connect((server0, port))
s1.connect((server1, port))

### Send data

In [ ]:
msg0 = pickle.dumps(d['A'])
msg1 = pickle.dumps(d['B'])

In [ ]:
# send it
s0.send(msg0)
s1.send(msg1)

In [ ]:
# close sockets
s0.close()
s1.close()

### Save timer

In [ ]:
# timer
end = time.time()
df_timer = pd.DataFrame(data={'phase': '1: Secret Sharing',
                              'timestamp': str(datetime.utcnow()),
                              'n_secrets': len(d),
                              'seconds': [round(end-start,2)]})

with open('data/timer.csv', 'a') as f:
    df_timer.to_csv(f, header=False)